In [3]:
import pandas as pd
import re
import stanza
import nltk
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic

/root/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# ============================
# 1. Инициализация инструментов
# ============================

# Скачиваем ресурсы
nltk.download("stopwords")
stanza.download("ru")

# Стоп-слова
ru_stopwords = set(stopwords.words("russian"))

# Stanza NLP-пайплайн с GPU
nlp = stanza.Pipeline(lang="ru", processors="tokenize,pos,lemma", use_gpu=True)
POS_ALLOWED = {"NOUN", "VERB", "ADJ", "ADV"}

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2025-05-22 13:07:05 INFO: Downloaded file to /root/stanza_resources/resources.json
2025-05-22 13:07:05 INFO: Downloading default packages for language: ru (Russian) ...
2025-05-22 13:07:09 INFO: File exists: /root/stanza_resources/ru/default.zip
2025-05-22 13:07:16 INFO: Finished downloading models and saved to /root/stanza_resources
2025-05-22 13:07:16 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-05-22 13:07:17 INFO: Downloaded file to /root/stanza_resources/resources.json
2025-05-22 13:07:17 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm 

In [26]:
# ============================
# 2. Предобработка: лемматизация и фильтрация
# ============================

def clean_text_stanza(text: str) -> str:
    """Очистка, лемматизация, удаление стоп-слов, POS-фильтрация и фильтр по длине"""
    text = re.sub(r"[^А-Яа-яЁё\s]", " ", str(text).lower())
    doc = nlp(text)
    lemmas = []
    for sentence in doc.sentences:
        for word in sentence.words:
            if (
                word.lemma
                and word.lemma not in ru_stopwords
                and word.upos in POS_ALLOWED
                and len(word.lemma) >= 3
            ):
                lemmas.append(word.lemma)
    return " ".join(lemmas)


In [27]:
# ============================
# 3. Загрузка данных
# ============================

df_with_txts=pd.read_parquet('/root/my project ver 2/full_size_markuped_df_hq.parquet')
df=df_with_txts[df_with_txts['predicted_label']==1]

documents = df["Текст трека"].astype(str).to_list()

In [28]:
# Лемматизация всех текстов
preprocessed_docs = []
for doc in documents:
    try:
        preprocessed_docs.append(clean_text_stanza(doc))
    except Exception:
        preprocessed_docs.append("")

In [29]:
preprocessed_docs

['бояться бояться главний слушать схотеть дать сильный главное сказать папирос эскимо порвать человек чуять прогинать понять гневно бушевать гора река замирать волна пальма танцевать море водопад коралл задавать ритм страза природа браз прикотиться бардак человек жизнь бывать раскидать грабли мирный идущий воин попутный творить мир зло война братик бабло политика послать нах заполнять пробел закрывать громко дверь любопытный гиена мысленно сужать веко накидать наболтать ахееть парабол давать посадить измена казаться невзъебенный проблема бля забивать разжигать улыбаться друг мир машь путь дорога унестись нога небо восход бывать бог хоровод водить алладина любить дым выпускать ветер улетать путь дорога унестись нога небо восход бывать бог хоровод водить алладина любить дым выпускать ветер улетать рэп чистый сорт марихуана накрывать жестко отголоска старый школа мясо брат новый рэп полный бардак запустить кишлак ужас мрак пятка блестеть факел гореть волк кричать рупор рубить паранойя гна

In [30]:
# ============================
# 4. Подготовка эмбеддинговой модели
# ============================

embedding_model = SentenceTransformer("ai-forever/sbert_large_nlu_ru")

# ============================
# 5. Векторизатор для c-TF-IDF
# ============================

vectorizer_model = CountVectorizer()
vectorizer_model.fit(preprocessed_docs)

CountVectorizer()

In [35]:
# ========== CountVectorizer с фильтрацией и кастомным токенизатором ==========

def stanza_tokenizer_for_vectorizer(text):
    try:
        return clean_text_stanza(text).split()
    except:
        return []

vectorizer_model = CountVectorizer(
    tokenizer=stanza_tokenizer_for_vectorizer,
    min_df=10,
    max_df=0.5,
)

In [36]:
# ============================
# 6. Обучение BERTopic
# ============================

topic_model = BERTopic(
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    language="multilingual",
    calculate_probabilities=True,
    verbose=True
)

topics, probs = topic_model.fit_transform(documents)

2025-05-21 12:56:53,417 - BERTopic - Embedding - Transforming documents to embeddings.


Batches: 100%|██████████| 501/501 [02:26<00:00,  3.42it/s]
2025-05-21 12:59:22,201 - BERTopic - Embedding - Completed ✓
2025-05-21 12:59:22,203 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-21 12:59:28,945 - BERTopic - Dimensionality - Completed ✓
2025-05-21 12:59:28,948 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-21 12:59:30,106 - BERTopic - Cluster - Completed ✓
2025-05-21 12:59:30,113 - BERTopic - Representation - Fine-tuning topics using representation models.


ValueError: max_df corresponds to < documents than min_df

In [34]:
# ============================
# 8. Визуализация (опционально)
# ============================

topic_model.visualize_barchart(top_n_topics=10).show()
topic_model.visualize_topics().show()

In [ ]:
# ============================
# 7. Анализ и вывод тем
# ============================

topic_info = topic_model.get_topic_info()
print(topic_info.head())

for topic_id in topic_info.Topic.unique():
    if topic_id == -1:
        continue
    top_words = [word for word, _ in topic_model.get_topic(topic_id)[:5]]
    print(f"Тема {topic_id}: {', '.join(top_words)}")

### Попробуем сделать BERTOPIC новый

In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import os
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

import torch

from umap import UMAP
from hdbscan import HDBSCAN
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

nltk.download('stopwords')

/root/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df_with_txts=pd.read_parquet('/root/my project ver 2/full_size_markuped_df_hq.parquet')
df=df_with_txts[df_with_txts['predicted_label']==1]

documents = df["Текст трека"].astype(str).to_list()

In [10]:
umap_model = UMAP(n_neighbors=12, n_components=5, metric='cosine', low_memory=False)
vectorizer_model = CountVectorizer(stop_words=stopwords.words('russian') + stopwords.words('english')+['это', 'ха', 'ya', 'ла', 'la', 'та'])
hdbscan_model = HDBSCAN(min_cluster_size=35, min_samples=15, metric='euclidean', prediction_data=True)
#embedding_model = SentenceTransformer("sentence-transformers/LaBSE")

topic_model = BERTopic(umap_model=umap_model,
                       vectorizer_model=vectorizer_model, 
                       hdbscan_model=hdbscan_model,
                       n_gram_range=(1,3),
#                       embedding_model=embedding_model,
                       nr_topics=150, top_n_words=10, language='multilingual', verbose=True).fit(documents)

2025-05-22 14:17:57,640 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 501/501 [00:14<00:00, 33.71it/s]
2025-05-22 14:18:18,468 - BERTopic - Embedding - Completed ✓
2025-05-22 14:18:18,470 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-22 14:18:24,941 - BERTopic - Dimensionality - Completed ✓
2025-05-22 14:18:24,943 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-22 14:18:25,974 - BERTopic - Cluster - Completed ✓
2025-05-22 14:18:25,976 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-05-22 14:18:29,839 - BERTopic - Representation - Completed ✓
2025-05-22 14:18:29,849 - BERTopic - Topic reduction - Reducing number of topics
2025-05-22 14:18:29,850 - BERTopic - Topic reduction - Number of topics (150) is equal or higher than the clustered topics(20).
2025-05-22 14:18:29,851 - BERTopic - Representation - Fine-tuning topics using representation mod

In [11]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,12011,-1_всё_тебе_эй_сука,"[всё, тебе, эй, сука, хочу, просто, мои, снова...","[Скорее бы весна, я тебя обниму\nПрижму к себе..."
1,0,69,0_дым_курю_shmouk_никотин,"[дым, курю, shmouk, никотин, курим, покурить, ...","[А я буду как дым, буду как дым,\nбуду как дым..."
2,1,98,1_око_черепа_crew_dead,"[око, черепа, crew, dead, демоны, моей, plug, ...","[Снова жажда крови на клинке, да, я готовый уб..."
3,2,303,2_всё_нам_мир_тебе,"[всё, нам, мир, тебе, лишь, жизнь, свет, небо,...","[Эй\nЭй, yeah, yeah, uh, uh, yeah, yeah\n\nЛиш..."
4,3,82,3_ремонту_жако_дойчланд_эй,"[ремонту, жако, дойчланд, эй, tag, па, всё, те...","[Давай крути, крути его, прямо до отсечки\n\nП..."
5,4,127,4_нам_всё_эй_жизнь,"[нам, всё, эй, жизнь, героиновый, ебано, время...","[алло, чё трубки не берёшь, гасишься?\nВосточн..."
6,5,558,5_её_хочет_хочу_сука,"[её, хочет, хочу, сука, мной, любит, всё, секс...","[Ты её потерял, телефон выключен, где она?\nВо..."
7,6,38,6_времена_рухнет_тап_всё,"[времена, рухнет, тап, всё, сошел, наглухо, пр...","[Буду незрим, когда дождь смоет грим\nРухнет к..."
8,7,39,7_пены_витамины_воду_drugs,"[пены, витамины, воду, drugs, папа, pill, фарм...",[Я начинаю свой день с хапки и хавки\nПапа на ...
9,8,90,8_outsider_тебе_снова_хочу,"[outsider, тебе, снова, хочу, тобой, коноплюху...","[И он снова пропал, так будешь ждать\nОказалас..."


In [12]:
topic_model.visualize_hierarchy()

In [13]:
topic_model.visualize_topics()

In [14]:
topics, probs = topic_model.transform(df['Текст трека'])

Batches: 100%|██████████| 501/501 [00:15<00:00, 31.54it/s]
2025-05-22 14:20:31,852 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2025-05-22 14:20:31,896 - BERTopic - Dimensionality - Completed ✓
2025-05-22 14:20:31,897 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2025-05-22 14:20:32,621 - BERTopic - Cluster - Completed ✓


In [15]:
df['topic'] = topics
df['topic_keywords'] = df['topic'].apply(
    lambda x: ", ".join([word for word, _ in topic_model.get_topic(x)]) if x != -1 else "None"
)
df['topic_probability'] = probs
df

,Исполнитель,Трек,Собран,Жанр,Доп. информация,Фит,Совместный релиз,Альбом,Год выхода,Текст трека,Языки в тексте,Части текста,Обработанные части,Очищенные леммы,predicted_label,confidence,prob_class_1,topic,topic_keywords,topic_probability
original_index,,,,,,,,,,,,,,,,,,,,
https://music.yandex.ru/album/21509172/track/101776208,SaNni,Где бывают боги,1.0,русский рэп,Нет,Нет,Несовместный релиз,Сингл,2019.0,"Это зачем?\nЧтоб боялись\nКого боятся, тот и г...","[ru, es, sr]","[Это зачем?\nЧтоб боялись\nКого боятся, тот и ...","[[это, зачем, ?, чтобы, бояться, кто, бояться,...","[[это, бояться, бояться, главний, слушать, схо...",1,0.983454,0.983454,-1,None,0.0
https://music.yandex.ru/album/20685194/track/99172841,SaNni,Моя,1.0,русский рэп,Нет,Нет,Alett,Сингл,2022.0,"Снова заберу тебя с собой, будто под водой\nНе...",[ru],"[Снова заберу тебя с собой, будто под водой\nН...","[[снова, забрать, ты, с, себя, ,, будто, под, ...","[[снова, забрать, вода, хватать, кислород, зна...",1,0.958730,0.958730,-1,None,0.0
https://music.yandex.ru/album/25088929/track/111941871,Klarko,Этому сну не стать вещим,1.0,русский рэп,Нет,Нет,Несовместный релиз,Сингл,2023.0,Пятьсот килоджоулей я сжег романтично\nДымит d...,[ru],[Пятьсот килоджоулей я сжег романтично\nДымит ...,"[[пятьсот, килоджоуль, я, сжечь, романтичный, ...","[[пятьсот, килоджоуль, сжечь, романтичный, Дым...",1,0.983501,0.983501,-1,None,0.0
https://music.yandex.ru/album/9269095/track/60155140,АЛМО,Сонный паралич,1.0,русский рэп,Нет,Нет,Несовместный релиз,Сингл,2019.0,Руки в волосах твоих пропахли керосином\nЯ пок...,[ru],[Руки в волосах твоих пропахли керосином\nЯ по...,"[[рука, в, волос, твой, пропахнуть, керосин, я...","[[рука, волос, твой, пропахнуть, керосин, поки...",1,0.766326,0.766326,2,"всё, нам, мир, тебе, лишь, жизнь, свет, небо, ...",1.0
https://music.yandex.ru/album/4229094/track/34362334,ИЧИ,Обратно,1.0,русский рэп,Нет,"ATL, ИЧИ",Несовместный релиз,Лимб,2017.0,"Мама, роди меня обратно\n\nЯ пожалел об этом т...",[ru],"[Мама, роди меня обратно, Я пожалел об этом ты...","[[мама, ,, родить, я, обратно], [я, жалеть, о,...","[[мама, родить, обратно], [жалеть, это, тысяче...",1,0.988983,0.988983,-1,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://music.yandex.ru/album/5988732/track/44620415,Слава КПСС,Скорпион,1.0,русский рэп,Нет,Нет,ЗАМАЙ,ОВЕРХАЙП 3,2018.0,"Яд!\nНаши темы — токсик, как яд\nМы в системе ...","[mk, ru, es, uk]","[Яд!\nНаши темы — токсик, как яд\nМы в системе...","[[яд, !, наш, тема, —, токсик, ,, как, яд, мы,...","[[яд, наш, тема, токсик, яд, система, против, ...",1,0.989701,0.989701,3,"ремонту, жако, дойчланд, эй, tag, па, всё, теб...",1.0
https://music.yandex.ru/album/7826259/track/54322612,LKN,яснопонятноок,1.0,русский рэп,Нет,Нет,Несовместный релиз,Сингл,2019.0,"Ясно понятно, ок\nТупая сука ведет диалог, ок\...",[ru],"[Ясно понятно, ок\nТупая сука ведет диалог, ок...","[[ясно, понятный, ,, ок, тупой, сука, вести, д...","[[ясно, понятный, ок, тупой, сука, вести, диал...",1,0.983381,0.983381,15,"тебе, снова, нам, сука, любовь, нахуй, мои, пр...",1.0
https://music.yandex.ru/album/11864118/track/70310541,XANSY,Тактический дробовик,1.0,рэп и хип-хоп,Нет,Нет,Несовместный релиз,Повесть временных треп,2020.0,вроде\n\nон пытается быть лучше\nтут старания ...,[ru],"[вроде, он пытается быть лучше\nтут старания н...","[[], [он, пытаться, быть, хороший, тут, старан...","[[], [пытаться, хороший, старание, напрасный, ...",1,0.512997,0.512997,15,"тебе, снова, нам, сука, любовь, нахуй, мои, пр...",1.0


In [16]:
df.to_parquet('df_with_drugs_topic_modeled_ver2.parquet')